In [ ]:
from autumn.core.project.project import get_project
from matplotlib import pyplot 
import pandas as pd
from autumn.settings.constants import COVID_BASE_DATETIME
from autumn.models.sm_sir.parameters import Parameters
from autumn.core import inputs
from summer.utils import ref_times_to_dti
import datetime
from autumn.core.inputs.database import get_input_db

In [ ]:
project = get_project("sm_sir", "wpro_singapore")
iso3 = "SGP"
country = "Singapore"

In [ ]:
custom_params = project.param_set.baseline 
model = project.run_baseline_model(custom_params)
derived_df = model.get_derived_outputs_df()

In [ ]:
 def get_actual_vaccination(iso3):

    input_db = get_input_db()
    vacc_data = input_db.query(table_name='owid', conditions= {"iso_code": iso3}, columns=["date", "people_fully_vaccinated_per_hundred"])
    vacc_data.dropna(inplace=True)
    return vacc_data

In [ ]:
vaccine_data = get_actual_vaccination(iso3)
df = vaccine_data.reset_index(drop=True)
print(df)
df.dtypes # to check the data types
df['date'] =pd.to_datetime(df['date']) # to convert to date time index to align with modelled derived_df data type

In [ ]:

vacc_strata = ["vaccinated", "unvaccinated"]
fig, axis = pyplot.subplots(1, 1, figsize=(12, 8))

# Loop over the vaccination types being implemented in the model
lower_value = [0.] * len(derived_df)
for stratum in vacc_strata:
    working_value = derived_df[f"prop_immune_{stratum}"]
    upper_value = lower_value + working_value
    axis.fill_between(derived_df.index, lower_value, upper_value, label=stratum)
    lower_value = upper_value

# import matplotlib.ticker as ticker
axis.plot(df.loc[:,"date"], df.loc[:,"people_fully_vaccinated_per_hundred"]/100, color="k", label="actual vaccination")


axis.legend(fontsize = 16)
axis.tick_params(axis="x", labelrotation=45, labelsize =15)
axis.tick_params(axis="y", labelsize =15)

# lower_lim = max(derived_df.index[0],df.date[0])
# upper_lim = min(derived_df.index[-1],df.date[df.index[-1]])

# axis.set_xlim(lower_lim, upper_lim)
axis.set_title(f"{country}")

fig.show()